# AWS Cost Analysis

This notebook demonstrates how to analyze AWS cost data using the CloudCostAI platform.

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add project root to path
sys.path.append('..')

# Import our modules
from src.collectors.aws import AWSCostCollector
from src.utils.helpers import generate_date_range

## Load Sample Data

For demonstration purposes, we'll load sample data from a CSV file.

In [ ]:
# Load sample data
df = pd.read_csv('../data/sample_billing_data.csv')

# Filter for AWS data only
aws_df = df[df['provider'] == 'AWS'].copy()

# Convert date to datetime
aws_df['date'] = pd.to_datetime(aws_df['date'])

# Display the first few rows
aws_df.head()

## Analyze Cost Trends

In [ ]:
# Group by date and service
service_costs = aws_df.pivot_table(index='date', columns='service', values='amount', aggfunc='sum')

# Plot service costs over time
plt.figure(figsize=(12, 6))
service_costs.plot(kind='bar', stacked=True, ax=plt.gca())
plt.title('AWS Costs by Service Over Time')
plt.xlabel('Date')
plt.ylabel('Cost (USD)')
plt.xticks(rotation=45)
plt.legend(title='Service')
plt.tight_layout()
plt.show()

## Service Cost Distribution

In [ ]:
# Calculate total cost by service
service_totals = aws_df.groupby('service')['amount'].sum().sort_values(ascending=False)

# Plot pie chart
plt.figure(figsize=(10, 8))
plt.pie(service_totals, labels=service_totals.index, autopct='%1.1f%%', startangle=90)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.title('AWS Cost Distribution by Service')
plt.show()

## Monthly Cost Growth

In [ ]:
# Calculate monthly totals
monthly_totals = aws_df.groupby(pd.Grouper(key='date', freq='M'))['amount'].sum()

# Calculate month-over-month growth
monthly_growth = monthly_totals.pct_change() * 100

# Plot monthly growth
plt.figure(figsize=(12, 6))
monthly_growth.plot(kind='bar', color=monthly_growth.map(lambda x: 'green' if x < 0 else 'red'))
plt.title('AWS Month-over-Month Cost Growth (%)')
plt.xlabel('Month')
plt.ylabel('Growth (%)')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Using the AWS Cost Collector

In a real scenario, you would use the AWSCostCollector to fetch actual data from AWS Cost Explorer API.

In [ ]:
# Initialize the collector
# Note: This requires proper AWS credentials to be configured
collector = AWSCostCollector()

# Get dates for the last 3 months
start_date, end_date = generate_date_range(3)

# Uncomment to fetch actual AWS cost data
# actual_aws_data = collector.get_cost_and_usage(start_date, end_date)
# actual_aws_data.head()

## Idle Resource Analysis

In [ ]:
# Get idle resources
idle_resources = collector.get_idle_resources()

# Display idle resources
idle_resources

## Conclusion

This notebook demonstrates basic AWS cost analysis using the CloudCostAI platform. In a real-world scenario, you would:

1. Connect to actual AWS Cost Explorer API
2. Analyze trends over longer periods
3. Identify cost optimization opportunities
4. Generate actionable recommendations